##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/tensorflow_text/intro"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/examples/intro.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/examples/intro.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/examples/intro.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# TF.Text

## 介绍

TensorFlow Text提供了一组与文本相关的类和操作，他们可以在TensorFlow 2.0中被使用。这个库可以执行基于文本模型需要的预处理，还包括一些对序列建模有用的特性，并且这些特性在core TensorFlow中没有被提供。
在文本预处理中使用这些操作的好处是它们是在TensorFlow图中完成的。您不需要担心训练中的词语切分与推理时的词语切分不同，也不需要担心管理预处理脚本。

## 即可执行（Eager Execution）

TensorFlow text需要TensorFlow 2.0，并且完全兼容Eager模式和图形模式。

---

注意: 在极少数情况下，此导入可能在查找TF库时失败。请重置运行时并重新运行上面的pip安装。

In [2]:
!pip install -q tensorflow-text

You should consider upgrading via the '/tmpfs/src/tf_docs_env/bin/python -m pip install --upgrade pip' command.


In [3]:
import tensorflow as tf
import tensorflow_text as text

## 统一的字符编码标准（Unicode）

大多数操作都希望字符串是UTF-8格式。如果使用不同格式的编码，可以使用core tensorflow的转换操作将它转换为UTF-8格式。如果输入无效，也可以使用相同的操作将字符串强制转化为结构上有效的UTF-8格式。

In [4]:
docs = tf.constant([u'Everything not saved will be lost.'.encode('UTF-16-BE'), u'Sad☹'.encode('UTF-16-BE')])
utf8_docs = tf.strings.unicode_transcode(docs, input_encoding='UTF-16-BE', output_encoding='UTF-8')

## 词语切分（Tokenization）

词语切分是将一个字符串分解成token的过程。通常这些token是单词、数字、/或标点符号。
    主要接口是Tokenizer和TokenizerWithOffsets，它们分别有一个方法tokenize和tokenize_with_offsets。现在有多个分词器可用。它们都实现了TokenizerWithOffsets(扩展了Tokenizer)，其中还包括一个将字节偏移量放入原始字符串的选项。这允许调用者知道创建token原始字符串中的字节。
### 空白分词器（WhitespaceTokenizer）

这是一个基本的分词器，它在ICU定义的空白字符上拆分UTF-8字符串（比如：空格，制表符，换行)。


In [5]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


[[b'everything', b'not', b'saved', b'will', b'be', b'lost.'], [b'Sad\xe2\x98\xb9']]


### 统一字符编码脚本的分词器（UnicodeScriptTokenizer）

 这个分词器可以根据Unicode脚本边界分割UTF-8字符串。使用的脚本代码对应(ICU)的UScriptCode值。详情请看：http://icu-project.org/apiref/icu4c/uscript_8h.html

 在实践中，这类似于WhitespaceTokenizer，最明显的区别在于它将标点符号(USCRIPT_COMMON)从语言文本中分离出来(例如。使用拉丁语，使用西里尔字母等)，同时也将语言文本彼此分开。

In [6]:
tokenizer = text.UnicodeScriptTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

[[b'everything', b'not', b'saved', b'will', b'be', b'lost', b'.'], [b'Sad', b'\xe2\x98\xb9']]


### 统一的字符编码标准切片（Unicode split）

在对没有空格的语言进行分词时，通常使用字符分割，这可以使用core中的unicode_split操作来完成。

In [7]:
tokens = tf.strings.unicode_split([u"仅今年前".encode('UTF-8')], 'UTF-8')
print(tokens.to_list())

[[b'\xe4\xbb\x85', b'\xe4\xbb\x8a', b'\xe5\xb9\xb4', b'\xe5\x89\x8d']]


### 偏移量（offset）

在对字符串进行分词时，通常需要知道token来自原始字符串中的何处。因此，每个实现TokenizerWithOffsets的分词器都有一个tokenize_with_offsets方法，该方法将返回字节偏移量和token。offset_starts列出了原始字符串中每个token开始的字节，而offset_limits列出了每个token结束的字节。

In [8]:
tokenizer = text.UnicodeScriptTokenizer()
(tokens, offset_starts, offset_limits) = tokenizer.tokenize_with_offsets(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())
print(offset_starts.to_list())
print(offset_limits.to_list())

[[b'everything', b'not', b'saved', b'will', b'be', b'lost', b'.'], [b'Sad', b'\xe2\x98\xb9']]
[[0, 11, 15, 21, 26, 29, 33], [0, 3]]
[[10, 14, 20, 25, 28, 33, 34], [3, 6]]


### TF.Data Example

分词器可以在tf.data接口的支持下工作的非常高效，下面是个简单的示例。

In [9]:
docs = tf.data.Dataset.from_tensor_slices([['Never tell me the odds.'], ["It's a trap!"]])
tokenizer = text.WhitespaceTokenizer()
tokenized_docs = docs.map(lambda x: tokenizer.tokenize(x))
iterator = iter(tokenized_docs)
print(next(iterator).to_list())
print(next(iterator).to_list())

[[b'Never', b'tell', b'me', b'the', b'odds.']]
[[b"It's", b'a', b'trap!']]


## 其他的文本操作（Other Text Ops）

tf.text包中还有另外一些有用的预处理操作。我们将在下面回顾一些。

### Wordshape

在一些自然语言理解模型中常用的一个特性是查看文本字符串是否具有某个属性。例如，断句模型可能包含检查单词大小写的属性，或者标点字符是否位于字符串末尾的属性。

Wordshape定义了各种有用的基于正则表达式的helper函数，用于匹配输入文本中的各种相关模式。这里有几个例子。

In [10]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])

# Is capitalized?
f1 = text.wordshape(tokens, text.WordShape.HAS_TITLE_CASE)
# Are all letters uppercased?
f2 = text.wordshape(tokens, text.WordShape.IS_UPPERCASE)
# Does the token contain punctuation?
f3 = text.wordshape(tokens, text.WordShape.HAS_SOME_PUNCT_OR_SYMBOL)
# Is the token a number?
f4 = text.wordshape(tokens, text.WordShape.IS_NUMERIC_VALUE)

print(f1.to_list())
print(f2.to_list())
print(f3.to_list())
print(f4.to_list())

[[True, False, False, False, False, False], [True]]
[[False, False, False, False, False, False], [False]]
[[False, False, False, False, False, True], [True]]
[[False, False, False, False, False, False], [False]]


### N-grams & Sliding Window

N-grams是给定一个滑动窗口大小为n的连续的单词。当组合token时，支持三种缩减机制。对于文本，您可能希望使用相互追加字符串的Reduction.STRING_JOIN方法。默认分隔符是空格，但可以使用string_separater参数更改。

另外两种最常用于数值的缩减方法，是Reduction.SUM和Reduction.MEAN。

In [11]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])

# Ngrams, in this case bi-gram (n = 2)
bigrams = text.ngrams(tokens, 2, reduction_type=text.Reduction.STRING_JOIN)

print(bigrams.to_list())

[[b'Everything not', b'not saved', b'saved will', b'will be', b'be lost.'], []]
